In [38]:
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

plt.style.use('seaborn-darkgrid')

In [ ]:
from statsmodels.tsa.ardl import ardl_select_order


In [39]:
input_dir = '../../data/final'

df = pd.read_csv(os.path.join(input_dir, 'df_final_geo_518_2.csv'))

/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_4100/1863682907.py:3: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(input_dir, 'df_final_geo_518_2.csv'))


In [40]:
# describe df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7053009 entries, 0 to 7053008
Data columns (total 30 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Unnamed: 0           int64  
 1   index_x              int64  
 2   datetime             object 
 3   Location name_x      object 
 4   ESMI_ID              float64
 5   District             object 
 6   State                object 
 7   Category             object 
 8   Connection Type_x    object 
 9   Lat                  float64
 10  Lon                  float64
 11  t2m                  float64
 12  u10                  float64
 13  v10                  float64
 14  tp                   float64
 15  date                 int64  
 16  time                 int64  
 17  station_id           int64  
 18  index_y              int64  
 19  pct_blackout         float64
 20  wind_speed           float64
 21  Connection Type_y    object 
 22  climate_zone_DN      float64
 23  climate_zone_name    object 
 24

In [41]:
df.columns

Index(['Unnamed: 0', 'index_x', 'datetime', 'Location name_x', 'ESMI_ID',
       'District', 'State', 'Category', 'Connection Type_x', 'Lat', 'Lon',
       't2m', 'u10', 'v10', 'tp', 'date', 'time', 'station_id', 'index_y',
       'pct_blackout', 'wind_speed', 'Connection Type_y', 'climate_zone_DN',
       'climate_zone_name', 'climate_zone_code', 'tree2000', 'gain',
       'lossyear', 'geometry', 'climate_zone_code_2'],
      dtype='object')

In [42]:

df['station_id'].value_counts()

531    78016
530    76734
529    76427
533    73466
532    69707
535    68785
539    61533
534    57662
541    57120
540    49274
544    49128
542    48375
543    45069
545    42020
495    36069
468    36047
44     35909
361    35875
464    35847
252    35832
251    35796
432    35427
215    35295
356    35219
172    35162
32     34912
158    34798
538    34715
48     34650
243    34611
81     34481
223    33674
70     33286
13     32985
339    32530
311    32190
195    32128
274    31867
340    31208
146    31140
186    30549
287    30248
431    30093
12     29683
498    29586
424    29211
425    29179
289    29124
17     29087
262    28908
426    28738
422    28644
313    28548
305    28539
194    28495
427    28429
83     28403
196    28188
376    28161
53     28159
25     28138
508    28030
36     27718
250    27659
97     27586
217    27492
458    27484
278    27429
214    27400
437    27371
355    26856
479    26651
242    26597
207    26058
140    26034
403    26024
212    25870

In [43]:
# select station_id 377

df_377 = df[df['station_id'] == 377]

# turn to a time series, use datetime as index
df_377['datetime'] = pd.to_datetime(df_377['datetime'])



/var/folders/38/ttqg2y215g16g2ng7jd502_c0000gn/T/ipykernel_4100/1561053690.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_377['datetime'] = pd.to_datetime(df_377['datetime'])


In [ ]:

def testMaxLagAndTime(df, maxlag):
    print('Begin testing...')
    print(f'testing maxlag: {maxlag}')
    time_start = pd.Timestamp.now()
    
    order_selected= ardl_select_order(df['pct_blackout'],	exog=df[[
                                   't2m', 'wind_speed', 'tp']], maxlag=maxlag, ic='aic', maxorder=maxlag).model.fit()
    
    time_selected = pd.Timestamp.now()
    print(f'time elapsed for selecting order: {time_selected-time_start}')
    
    test_model = order_selected.model.fit()
    time_fit = pd.Timestamp.now()
    print(f'time elapsed for fitting model: {time_fit-time_selected}')
    
    time_end = pd.Timestamp.now()
    print(f'time elapsed for the whole process: {time_end-time_start}')
    print('End testing...')
    print('model summary:')
    print(test_model.summary())
    return test_model

In [ ]:
test_model = testMaxLagAndTime(df_377, 10)
# takes 30 seconds

testing maxlag: 10
Begin testing


/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:00:32.803390
time elapsed for fitting model: 0 days 00:00:00.003176
time elapsed for the whole process: 0 days 00:00:32.806670
End testing
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                 5410
Model:                  ARDL(2, 0, 5)   Log Likelihood                 281.921
Method:               Conditional MLE   S.D. of innovations              0.230
Date:                Wed, 11 Dec 2024   AIC                           -541.843
Time:                        12:49:14   BIC                           -469.291
Sample:                             5   HQIC                          -516.515
                                 5410                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
co

/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
test_model = testMaxLagAndTime(df_377, 24)
# takes 15 min

In [37]:
test_model = testMaxLagAndTime(df_377, 17)

testing maxlag: 17
Begin testing


/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/rainylty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


time elapsed for selecting order: 0 days 00:08:24.925978
time elapsed for fitting model: 0 days 00:00:00.003843
time elapsed for the whole process: 0 days 00:08:24.929960
End testing
model summary:
                              ARDL Model Results                              
Dep. Variable:           pct_blackout   No. Observations:                 5410
Model:                  ARDL(2, 0, 5)   Log Likelihood                 281.921
Method:               Conditional MLE   S.D. of innovations              0.230
Date:                Wed, 11 Dec 2024   AIC                           -541.843
Time:                        13:13:13   BIC                           -469.291
Sample:                             5   HQIC                          -516.515
                                 5410                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
co